<a href="https://colab.research.google.com/github/carlosarp21/final_ic/blob/dev/Proyecto_de_limpieza_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### INSTRUCCIONES

##### El archivo adjunto data-1.csv, contiene datos ordenados por fechas de dos variables: $ x1 $ y $ x2 $. De los cuales se sabe que ninguna observación puede ser menor o igual que cero. 

<br>

1. Separe las variables x1 y x2 en columnas diferentes.

2. Elija y justifique un método de imputación de datos para los datos perdidos y los datos menores o iguales que cero.

3. Encuentre un shapelet de longitud 365 y otro de longitud 1460 para cada variable.

4. Siendo el shapelet de 365 el identificador de la clase cero y el shapelet de 1460 el identificador de clase uno. Entrene un Árbol de decisión para clasificar ambas variables y mencione la frecuencia de cada una (En caso de traslape de clases, la clase cero tiene prioridad).

---

#### DESARROLLO

In [1]:
import pandas as pd

In [3]:
# Se inicia con la lectura del archivo
df = pd.read_csv('/content/data-1.csv')

##### Análisis exploratorio de los datos

In [13]:
# Exploración del df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6571 entries, 0 to 6570
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   nombre  6571 non-null   object 
 1   Fecha   6571 non-null   object 
 2   data    6362 non-null   float64
dtypes: float64(1), object(2)
memory usage: 154.1+ KB


In [12]:
# Resumen estadístico
df.describe()

,data
count,6362.000000
mean,20.972573
std,32.232027
min,-73.790000
25%,3.400000
50%,9.440000
75%,25.600000
max,299.740000


In [11]:
# Vista de los primero registros
df.head()

,nombre,Fecha,data
0,x1,2006-01-01,6.05
1,x1,2006-01-02,4.72
2,x1,2006-01-03,4.46
3,x1,2006-01-04,34.74
4,x1,2006-01-05,1.91


In [7]:
# Vista de los últimos registros
df.tail()

,nombre,Fecha,data
6566,x2,27/12/2014,-13.3
6567,x2,28/12/2014,16.2
6568,x2,29/12/2014,5.1
6569,x2,30/12/2014,4.9
6570,x2,31/12/2014,3.7


In [26]:
# Se observa que se cuenta con 6571 registros, en tres columnas. 
# En la columna *data* hay valores faltantes.
df[df.data.isnull()]

,nombre,Fecha,data
3053,x1,2014-05-12,NaN
3054,x1,2014-05-13,NaN
3055,x1,2014-05-14,NaN
3056,x1,2014-05-15,NaN
3068,x1,2014-05-27,NaN
...,...,...,...
5839,x2,27/12/2012,NaN
5840,x2,28/12/2012,NaN
5841,x2,29/12/2012,NaN
5842,x2,30/12/2012,NaN


In [30]:
# Conteo de valores únicos en cada columna
print(df.nombre.nunique(), df.Fecha.nunique(), df.data.nunique())

# nombre - 2 valores únicos
# fecha - 6571 valores únicos
# data - 3450 valores únicos

2 6571 3450


---

##### 1. Separación de variables.

In [33]:
### Se convierte el formato de fechas para tenerlo de manera uniforme
fx1 = pd.to_datetime(df['Fecha'], format='%Y-%m-%d', errors='coerce')
fx2 = pd.to_datetime(df['Fecha'], format='%d/%m/%Y', errors='coerce')
df['Fecha'] = fx1.combine_first(fx2)

# Cuando se uniforma el formato de fecha, los valores únicos de esta
# columna se reducen de 6571 a 3287 debido a que había fechas comunes
# pero que no se detectaban por la diferencia de formato.
df.Fecha.value_counts()

In [61]:
# Separación del df en función del valor de la columna nombre
dfx1 = df[df.nombre=='x1']
dfx2 = df[df.nombre=='x2']

# Merge de los dos df por columna de fecha
df2 = pd.merge(dfx1, dfx2, on='Fecha', suffixes=('_x1', '_x2'))

# Reorganización del nuevo df2
df2 = df2[['Fecha', 'nombre_x1', 'data_x1', 'nombre_x2', 'data_x2']]
df2

---

##### 2. Inputación de dato.

In [99]:
# Búqueda de datos faltantes en x1, el criterio será
# valores nulos o valores menores o iguales a cero.
df2[(df2.data_x1.isnull()) | (df2.data_x1<= 0)]

,Fecha,nombre_x1,data_x1,nombre_x2,data_x2
22,2006-01-23,x1,-3.81,x2,-2.81
41,2006-02-11,x1,-1.41,x2,8.98
61,2006-03-03,x1,-1.91,x2,-5.01
63,2006-03-05,x1,-1.76,x2,6.66
66,2006-03-08,x1,-0.26,x2,-1.13
...,...,...,...,...,...
3243,2014-11-21,x1,NaN,x2,2.80
3244,2014-11-22,x1,NaN,x2,9.50
3245,2014-11-23,x1,NaN,x2,6.90
3253,2014-12-01,x1,NaN,x2,-12.20


In [100]:
# Aplicando el mismo criterio para los valores de x2.
df2[(df2.data_x2.isnull()) | (df2.data_x2<= 0)]

,Fecha,nombre_x1,data_x1,nombre_x2,data_x2
1,2006-01-02,x1,4.72,x2,-13.57
8,2006-01-09,x1,13.90,x2,-4.86
12,2006-01-13,x1,13.21,x2,-0.90
15,2006-01-16,x1,9.77,x2,-1.30
22,2006-01-23,x1,-3.81,x2,-2.81
...,...,...,...,...,...
3266,2014-12-14,x1,19.12,x2,-0.70
3267,2014-12-15,x1,28.92,x2,-14.00
3269,2014-12-17,x1,19.37,x2,-2.40
3274,2014-12-22,x1,12.12,x2,-19.60
